In [1]:
import sys
sys.path.append('../src')

from time_series import (
    load_data
)


In [2]:
df = load_data('../data/data_original_m6.csv')
df.head()

,ABBV,ACN,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,...,XLC,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2022-02-01,125.672005,341.526031,81.144257,144.980698,116.244766,135.845093,295.972290,151.193497,225.756577,198.813828,...,73.001434,62.652004,37.921776,98.328560,159.164963,71.541222,63.674873,126.980408,182.482285,73.886658
2022-02-02,127.158058,347.372375,82.549454,146.648575,116.419128,137.952209,300.881073,150.612503,232.359009,193.332672,...,74.459702,62.853901,38.180336,99.012138,160.458786,72.398964,64.616043,128.688538,181.393707,73.694695
2022-02-03,129.093597,333.920837,82.834190,145.507904,112.476517,133.787003,292.273804,138.845505,231.047760,186.040558,...,69.478104,62.257393,37.758987,97.433182,155.803101,72.417839,64.327171,128.138443,175.842789,72.835434
2022-02-04,129.020187,335.805206,81.897385,146.725250,115.518234,132.865723,295.972290,157.639496,228.360596,182.360779,...,69.654274,63.230167,38.400589,96.383751,156.430374,71.531807,63.814636,127.675232,180.844467,74.416832
2022-02-07,130.744751,332.384338,82.485176,146.408951,116.690369,133.414566,299.632263,157.935501,225.654984,174.808609,...,68.440689,64.037735,38.515503,96.460770,155.391418,71.663765,63.721447,127.453300,180.471817,75.312653


In [6]:
import pandas as pd

# Load your dataset
# df = pd.read_csv("../data/data_original_m6.csv", parse_dates=['Date'], index_col='Date')
s = df['ABBV']  # Example stock


In [7]:
import pandas as pd
from typing import Union, Optional

def generate_lags(
    series: Union[pd.Series, pd.DataFrame],
    past_lags: int,
    future_lags: int,
    dropna: bool = True,
) -> pd.DataFrame:
    """
    Generate a lagged DataFrame from a univariate or multivariate time series.

    Parameters
    ----------
    series : pd.Series or pd.DataFrame
        Input time series (indexed by datetime).
    past_lags : int, default=1
        Number of past lags to include (y - 1, ..., y - p).
    future_lags : int, default=0
        Number of future lags to include (y + 1, ..., y + f).
    dropna : bool, default=False
        Whether to drop rows with NaNs created by shifting.

    Returns
    -------
    pd.DataFrame
        DataFrame with lagged values in the order:
        [y - p, ..., y - 1, y, y + 1, ..., y + f]
    """
    if isinstance(series, pd.Series):
        series = series.to_frame()

    # List to hold all shifted DataFrames
    dfs = []

    # Past lags (y - p to y - 1)
    for past in range(past_lags, 0, -1):
        lagged = series.shift(past)
        lagged.columns = [f"y-{past}" for _ in series.columns]
        dfs.append(lagged)

    # Current y
    current = series.copy()
    current.columns = ["y" for _ in series.columns]
    dfs.append(current)

    # Future lags (y + 1 to y + f)
    for future in range(1, future_lags + 1):
        lead = series.shift(-future)
        lead.columns = [f"y+{future}" for _ in series.columns]
        dfs.append(lead)

    # Concatenate all
    df = pd.concat(dfs, axis=1)

    return df.dropna() if dropna else df


In [8]:
import pandas as pd

# s = pd.Series(range(10,110, 10), name="y")
df = generate_lags(s, past_lags=2, future_lags=2)

display(df)


,y-2,y-1,y,y+1,y+2
Date,,,,,
2022-02-03,125.672005,127.158058,129.093597,129.020187,130.744751
2022-02-04,127.158058,129.093597,129.020187,130.744751,131.643707
2022-02-07,129.093597,129.020187,130.744751,131.643707,131.359375
2022-02-08,129.020187,130.744751,131.643707,131.359375,130.909882
2022-02-09,130.744751,131.643707,131.359375,130.909882,130.267746
...,...,...,...,...,...
2023-02-16,145.301041,143.844116,142.387222,144.082184,144.739227
2023-02-17,143.844116,142.387222,144.082184,144.739227,143.472748
2023-02-21,142.387222,144.082184,144.739227,143.472748,144.805893


In [9]:
df.columns

Index(['y-2', 'y-1', 'y', 'y+1', 'y+2'], dtype='object')

In [10]:
from sklearn.metrics import mean_absolute_percentage_error
import pandas as pd
from typing import List, Type
from sklearn.base import BaseEstimator

def fit_model_from_lagged(
    df: pd.DataFrame,
    lags: List[str],
    model_class: Type[BaseEstimator],
    **model_kwargs
) -> tuple[BaseEstimator, float]:
    """
    Fit a scikit-learn-like model using lagged features to predict the 'y' column,
    and evaluate it using Mean Absolute Percentage Error (MAPE).

    Args:
        df (pd.DataFrame): DataFrame containing the 'y' column and lag/lead features.
        lags (List[str]): List of column names to be used as input features.
        model_class (Type[BaseEstimator]): A scikit-learn-compatible model class.
        **model_kwargs: Additional keyword arguments passed to the model constructor.

    Returns:
        tuple[BaseEstimator, float]: A fitted model instance and its MAPE on training data.

    Raises:
        ValueError: If 'y' column or any lag feature is missing in the DataFrame.
    """
    if 'y' not in df.columns:
        raise ValueError("The DataFrame must contain a 'y' column as target.")

    missing_lags = [lag for lag in lags if lag not in df.columns]
    if missing_lags:
        raise ValueError(f"The following lag columns are missing: {missing_lags}")

    X = df[lags].copy().values
    y = df['y'].copy().values

    model = model_class(**model_kwargs)
    model.fit(X, y)

    y_pred = model.predict(X)
    mape = mean_absolute_percentage_error(y, y_pred)

    return model, mape


In [11]:
from sklearn.linear_model import LinearRegression

lags = df.columns

model, mape = fit_model_from_lagged(
    df=df,
    lags=lags,
    model_class=LinearRegression
)
print(mape)


6.034426818349943e-17
